In [4]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "提取事件信息。"},
        {"role": "user", "content": "艾丽斯和鲍勃星期五要去参加科学展览。"},
    ],
    response_format=CalendarEvent,
)


event = completion.choices[0].message.parsed
print(completion)
print(event)

ParsedChatCompletion[CalendarEvent](id='chatcmpl-9wllnffyHW4go3V6Mw1KYq3i0daXj', choices=[ParsedChoice[CalendarEvent](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[CalendarEvent](content='{"name":"科学展览","date":"星期五","participants":["艾丽斯","鲍勃"]}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=CalendarEvent(name='科学展览', date='星期五', participants=['艾丽斯', '鲍勃'])))], created=1723793679, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_48196bc67a', usage=CompletionUsage(completion_tokens=24, prompt_tokens=37, total_tokens=61))
name='科学展览' date='星期五' participants=['艾丽斯', '鲍勃']


# 案例

## 思维链
您可以要求模型以结构化、逐步的方式输出答案，引导用户逐步解决问题。

In [7]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "您是一位乐于助人的数学导师。逐步引导用户完成解决方案。"},
        {"role": "user", "content": "我如何解决8x+7=-23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed
print(math_reasoning)

steps=[Step(explanation='To isolate the term with the variable, subtract 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='This simplifies to 8x = -30. Now, the equation has the form 8x = -30 and we need to solve for x by diving both sides by 8.', output='8x = -30'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -30/8'), Step(explanation='Simplify the fraction by dividing both the numerator and the denominator by their greatest common divisor, which is 2.', output='x = -15/4')] final_answer='x = -15/4'


**结果如下**
steps=[
    Step(explanation='To isolate the term with the variable, subtract 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), 
    Step(explanation='This simplifies to 8x = -30. Now, the equation has the form 8x = -30 and we need to solve for x by diving both sides by 8.', output='8x = -30'), 
    Step(explanation='Divide both sides by 8 to solve for x.', output='x = -30/8'), 
    Step(explanation='Simplify the fraction by dividing both the numerator and the denominator by their greatest common divisor, which is 2.', output='x = -15/4')
    ] 
final_answer='x = -15/4'


## 结构化数据提取
您可以定义结构化字段，从非结构化输入数据（如研究论文）中提取。

In [9]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "您是结构化数据提取方面的专家。您将获得来自研究论文的非结构化文本，并应将其转换为给定的结构。"},
        {"role": "user", "content": "从非结构化输入生成结构化数据是当今应用中 AI 的核心应用场景之一。开发者利用 OpenAI API 构建强大的助手，这些助手能够通过函数调用获取数据并回答问题，提取结构化数据以进行数据录入，并构建多步骤的代理工作流程，使LLMs能够采取行动。开发者长期以来一直在通过开源工具、提示工程和反复重试请求来绕过LLMs在这一领域的限制，以确保模型输出符合与其系统互操作所需的格式。结构化输出通过约束 OpenAI 模型匹配开发者提供的模式，并通过训练我们的模型更好地理解复杂的模式，解决了这一问题。"}
    ],
    response_format=ResearchPaperExtraction,
)

research_paper = completion.choices[0].message.parsed
print(research_paper)

title='Structured Data Extraction with AI' authors=['John Doe', 'Jane Smith'] abstract='This paper discusses the core application of AI in structured data extraction, highlighting the use of OpenAI API by developers to build assistants that acquire data through function calls, answer questions, and perform multi-step proxy workflows. The challenges faced by developers in overcoming limitations of LLMs in this field are also addressed, along with the introduction of structured outputs that constrain model outputs to match developer-provided schemas.' keywords=['Structured Data Extraction', 'AI', 'OpenAI API', 'Interoperability', 'Model Constraints']


**结果如下**
title='Structured Data Extraction with AI' 
authors=['John Doe', 'Jane Smith'] 
abstract='This paper discusses the core application of AI in structured data extraction, highlighting the use of OpenAI API by developers to build assistants that acquire data through function calls, answer questions, and perform multi-step proxy workflows. The challenges faced by developers in overcoming limitations of LLMs in this field are also addressed, along with the introduction of structured outputs that constrain model outputs to match developer-provided schemas.' 
keywords=['Structured Data Extraction', 'AI', 'OpenAI API', 'Interoperability', 'Model Constraints']

## UI生成
你可以通过将 HTML 表示为带有约束的递归数据结构（如枚举）来生成有效的 HTML。

In [10]:
from enum import Enum
from typing import List
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class UIType(str, Enum):
    div = "div"
    button = "button"
    header = "header"
    section = "section"
    field = "field"
    form = "form"

class Attribute(BaseModel):
    name: str
    value: str

class UI(BaseModel):
    type: UIType
    label: str
    children: List["UI"] 
    attributes: List[Attribute]

UI.model_rebuild() # This is required to enable recursive types

class Response(BaseModel):
    ui: UI

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "你是一个UI生成器AI。将用户输入转换为UI。"},
        {"role": "user", "content": "制作用户资料表"}
    ],
    response_format=Response,
)

ui = completion.choices[0].message.parsed
print(ui)

ui=UI(type=<UIType.form: 'form'>, label='User Profile Form', children=[UI(type=<UIType.field: 'field'>, label='First Name', children=[], attributes=[Attribute(name='type', value='text'), Attribute(name='name', value='first_name'), Attribute(name='placeholder', value='Enter your first name')]), UI(type=<UIType.field: 'field'>, label='Last Name', children=[], attributes=[Attribute(name='type', value='text'), Attribute(name='name', value='last_name'), Attribute(name='placeholder', value='Enter your last name')]), UI(type=<UIType.field: 'field'>, label='Email Address', children=[], attributes=[Attribute(name='type', value='email'), Attribute(name='name', value='email'), Attribute(name='placeholder', value='Enter your email address')]), UI(type=<UIType.field: 'field'>, label='Phone Number', children=[], attributes=[Attribute(name='type', value='tel'), Attribute(name='name', value='phone'), Attribute(name='placeholder', value='Enter your phone number')]), UI(type=<UIType.field: 'field'>, lab

**结果如下**
``` json
{
    "type": "form",
    "label": "User Profile Form",
    "children": [
        {
            "type": "div",
            "label": "",
            "children": [
                {
                    "type": "field",
                    "label": "First Name",
                    "children": [],
                    "attributes": [
                        {
                            "name": "type",
                            "value": "text"
                        },
                        {
                            "name": "name",
                            "value": "firstName"
                        },
                        {
                            "name": "placeholder",
                            "value": "Enter your first name"
                        }
                    ]
                },
                {
                    "type": "field",
                    "label": "Last Name",
                    "children": [],
                    "attributes": [
                        {
                            "name": "type",
                            "value": "text"
                        },
                        {
                            "name": "name",
                            "value": "lastName"
                        },
                        {
                            "name": "placeholder",
                            "value": "Enter your last name"
                        }
                    ]
                }
            ],
            "attributes": []
        },
        {
            "type": "button",
            "label": "Submit",
            "children": [],
            "attributes": [
                {
                    "name": "type",
                    "value": "submit"
                }
            ]
        }
    ],
    "attributes": [
        {
            "name": "method",
            "value": "post"
        },
        {
            "name": "action",
            "value": "/submit-profile"
        }
    ]
}
```

## 审核
您可以将输入归类到多个类别中，这是进行内容审核的常见方式。

In [11]:
from enum import Enum
from typing import Optional
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class Category(str, Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "确定用户输入是否违反特定准则并解释是否违反。"},
        {"role": "user", "content": "我如何准备工作面试？"}
    ],
    response_format=ContentCompliance,
)

compliance = completion.choices[0].message.parsed
print(compliance)

is_violating=False category=None explanation_if_violating=None


**结果如下**
is_violating=False 
category=None 
explanation_if_violating=None